<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Modification of the fuselage length of an aircraft.

This notebook is based on the 3rd FAST-OAD-GA tutorial written by Aurélien. It aims to compare a reference architecture represented by the .XML file of an aircraft (Beechcraft 76, Cirrus 22, TBM-900) with a modified configuration. The modified architecture is produced by the FAST-OAD-GA module called modify_config which creates the corresponding .XML file. Then the .xml files of the reference aircraft and of the modified one go through all of the FAST-OAD-GA modules, with the generate_block_analysis function which calls each module. The variable viewer is called for both aircrafts after each module.

The modification implemented in this notebook is the increase of the fuselage length in order to add a row of passengers. The user has to define the multiplier of the span, and can also play on other parameters such as setting the engine position along the span.

The fastga code is from the root analysis_mode on GitHub.

Like in the tutorials, the structure of the notebook is based on data transfers between the data folder which stocks the .xml of the reference aircraft, and the workdir folder which stocks the .xml files passing through the modules of FAST-OAD-GA. After all the modules have been executed the final .xml files are placed in the output folder.

As the TBM-900 turbopropeller is still being modelized in FAST-OAD-GA, the Beechcraft 76 will be used for this analysis. There is no limit in the code for the increase in span, but some modules could have problems running with an extreme value (such as two times the original span).

## 1. Initialization of the work environment

In [1]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import fastga
import logging
from fastoad.gui import VariableViewer
from fastoad.io import VariableIO
import shutil

# Define relative path.
current_path = os.getcwd()
DATA_FOLDER_PATH = pth.join(current_path, 'data')
WORK_FOLDER_PATH = pth.join(current_path, 'workdir')
OUTPUT_FOLDER_PATH = pth.join(current_path, 'output')

# Clear work folder.
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)
os.mkdir(WORK_FOLDER_PATH)

# For using more screen width.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Final file names. These .xml files will be overwritten each time they pass through a module.
AIRCRAFT_REF_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_reference.xml')
AIRCRAFT_MOD_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_mod.xml')

# Copy the reference aircraft file in the files of the work folder.
shutil.copy(pth.join(DATA_FOLDER_PATH, 'beechcraft_76.xml'), AIRCRAFT_REF_FILE)
# shutil.copy(pth.join(DATA_FOLDER_PATH, 'beechcraft_76.xml'), AIRCRAFT_MOD_FILE)

Failed to import module fastga.models.performances.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.geometry.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.loops.unitary_tests.test_wing_area.py
Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.performances.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.load_analysis.fuselage.compute_bending_moment_reversed_old.py
Failed to import module fastga.

'C:\\Users\\Lucas\\FAST-OAD-GA\\FAST-GA\\src\\fastga\\notebooks\\Lucas\\workdir\\geometry_mod.xml'

## Propeller Computation

In [2]:
from fastga.models.aerodynamics.components.compute_propeller_aero import ComputePropellerPerformance

compute_propeller_ref = api_cs23.generate_block_analysis(
        ComputePropellerPerformance(),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

output_ref = compute_propeller_ref({})

# Open viewer
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

shutil.copy(AIRCRAFT_REF_FILE, AIRCRAFT_MOD_FILE)

Entering propeller computation


2 Explicit Output(s) in 'system.propeller_aero_conditions'

varname                               value     
------------------------------------  ----------
data:aerodynamics:propeller:mach      [0.]      
data:aerodynamics:propeller:reynolds  [1000000.]


0 Implicit Output(s) in 'system.propeller_aero_conditions'


2 Explicit Output(s) in 'system.propeller_aero_conditions'

varname                               value     
------------------------------------  ----------
data:aerodynamics:propeller:mach      [0.]      
data:aerodynamics:propeller:reynolds  [1000000.]


0 Implicit Output(s) in 'system.propeller_aero_conditions'




C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:478: RuntimeWarning: divide by zero encountered in true_divide
  (radius_max - radius) / radius * math.sqrt(1 + (omega * radius / (v_inf + v_i)) ** 2.0)
c:\users\lucas\fast-oad-ga\venvtest\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:358: UserWarning: Propeller not working in propulsive mode!
  warnings.warn("Propeller not working in propulsive mode!")
c:\users\lucas\fast-oad-ga\venvtest\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\Lucas\FAST

In [ ]:
OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref_fuselage_be76.xml')
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_fuselage_be76.xml')
shutil.copy(AIRCRAFT_REF_FILE, OUTPUT_REF_FILE)
shutil.copy(AIRCRAFT_MOD_FILE, OUTPUT_MOD_FILE)

## 2. Definition of the fuselage modifications from the reference architecture

Describes the different parameters the user has to choose in order to obtain the desired modified configuration. The function generate_block_analysis is then used twice to create the .xml file related to the modified configuration.

The same parameters have to be used for the ComputeConfigMod class and for the UpdateXML class.

In [3]:
# Fuselage modifications
added_length = 0    # If 0, the length of a row of seats will be taken
added_section_x_ratio_front = 0
added_section_x_ratio_rear = 1
added_pax = 2     # Integer stating the number of passengers added in the design mission. It will be added to the npax_design of the xml, so take care not to exceed the new npax_max.
added_luggage = 10     # float stating the mass of luggage added in the design mission

fuselage_mod_parameters = [added_length, added_section_x_ratio_front, added_section_x_ratio_rear, added_pax, added_luggage]

In [4]:
from fastga.models.modify_config import ComputeConfigMod


compute_modify = api_cs23.generate_block_analysis(
        ComputeConfigMod(fuselage_mod=fuselage_mod_parameters),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_mod = compute_modify({})

# Open viewer
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

In [5]:
from fastga.models.modify_config.update_XML import UpdateXML


compute_update = api_cs23.generate_block_analysis(
        UpdateXML(fuselage_mod=fuselage_mod_parameters),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_mod = compute_update({})

# Open viewer
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## Computation of the reference and the modified architectures

In [6]:
from fastga.models.geometry import GeometryFixedTailDistance as Geometry
from fastga.models.aerodynamics.aerodynamics import Aerodynamics
from fastga.models.weight.weight import Weight
from fastga.models.weight.mass_breakdown.update_mtow import UpdateMTOW
from fastga.models.performances.mission import Mission
from fastga.models.load_analysis.loads import Loads
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities

# engine_id = "fastga.wrapper.propulsion.basic_turboprop"
engine_id = "fastga.wrapper.propulsion.basicIC_engine"

# GEOMETRY
compute_geometry_ref = api_cs23.generate_block_analysis(
        Geometry(propulsion_id=engine_id),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

compute_geometry_mod = api_cs23.generate_block_analysis(
        Geometry(propulsion_id=engine_id),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_ref = compute_geometry_ref({})
output_mod = compute_geometry_mod({})


# AERODYNAMICS
compute_aero_ref = api_cs23.generate_block_analysis(
        Aerodynamics(
            propulsion_id=engine_id,
            use_openvsp=True,
            compute_mach_interpolation=True,
            compute_slipstream_cruise=True,
#             wing_airfoil_file="naca43013_3.af"
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

compute_aero_mod = api_cs23.generate_block_analysis(
        Aerodynamics(
            propulsion_id=engine_id,
            use_openvsp=True,
            compute_mach_interpolation=True,
            compute_slipstream_cruise=True,
#             wing_airfoil_file="naca43013_3.af"
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_ref = compute_aero_ref({})
output_mod = compute_aero_mod({})


# WEIGHT
compute_weight_ref = api_cs23.generate_block_analysis(
        Weight(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

compute_weight_mod = api_cs23.generate_block_analysis(
        Weight(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_ref = compute_weight_ref({})
output_mod = compute_weight_mod({})


# MTOW
var_inputs = ["data:mission:sizing:fuel"]

compute_mtow_ref = api_cs23.generate_block_analysis(
        UpdateMTOW(),
        var_inputs,
        str(AIRCRAFT_REF_FILE),
        True,
    )

compute_mtow_mod = api_cs23.generate_block_analysis(
        UpdateMTOW(),
        var_inputs,
        str(AIRCRAFT_MOD_FILE),
        True,
    )

inputs_dict = {"data:mission:sizing:fuel": (150., "kg")}

output_ref = compute_mtow_ref(inputs_dict)
output_mod = compute_mtow_mod(inputs_dict)


# PERFORMANCES
compute_sizing_ref = api_cs23.generate_block_analysis(
        Mission(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

compute_sizing_mod = api_cs23.generate_block_analysis(
        Mission(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_ref = compute_sizing_ref({})
output_mod = compute_sizing_mod({})


# LOAD ANALYSIS
compute_loads_ref = api_cs23.generate_block_analysis(
        Loads(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

compute_loads_mod = api_cs23.generate_block_analysis(
        Loads(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_ref = compute_loads_ref({})
output_mod = compute_loads_mod({})


# HANDLING QUALITIES
compute_hq_ref = api_cs23.generate_block_analysis(
        ComputeHandlingQualities(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

compute_hq_mod = api_cs23.generate_block_analysis(
        ComputeHandlingQualities(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_ref = compute_hq_ref({})
output_mod = compute_hq_mod({})

C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\load_analysis\fuselage\compute_bending_moment.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  load_case_array = np.array([[n_lift, lift_max_h], [n_emergency_landing, 0]])


In [15]:
OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_ref_fuselage_be76.xml')
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_fuselage_be76.xml')
shutil.copy(AIRCRAFT_REF_FILE, OUTPUT_REF_FILE)
shutil.copy(AIRCRAFT_MOD_FILE, OUTPUT_MOD_FILE)

'C:\\Users\\Lucas\\FAST-OAD-GA\\FAST-GA\\src\\fastga\\notebooks\\Lucas\\output\\output_mod_fuselage_be76.xml'

In [7]:
api_cs25.variable_viewer(AIRCRAFT_REF_FILE)

In [8]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

In [16]:
variables_ref = VariableIO(AIRCRAFT_REF_FILE).read()
wing_loading_ref = variables_ref["data:loads:wing_loading"].value[0]

variables_mod = VariableIO(AIRCRAFT_MOD_FILE).read()
wing_loading_mod = variables_mod["data:loads:wing_loading"].value[0]

print(wing_loading_ref, wing_loading_mod)

111.35119345717554 121.26088373463811


## Analysis of the wing span increase in order to get a lower wing loading

In [17]:
AIRCRAFT_SPAN_FILE = pth.join(WORK_FOLDER_PATH, 'aircraft_span_mod.xml')
shutil.copy(OUTPUT_MOD_FILE, AIRCRAFT_SPAN_FILE)

'C:\\Users\\Lucas\\FAST-OAD-GA\\FAST-GA\\src\\fastga\\notebooks\\Lucas\\workdir\\geometry_mod.xml'

In [ ]:
# Span modifications
span_length_multiplier = 1.1       # value by which the wing span is multiplied. If 1.0, the span doesn't change.
fixed_engine_position = True       # boolean stating if the engines stay at the same y-position along the span or if their y-ratio is conserved
fuel_added_part = False            # boolean stating if fuel tanks are added in the extended part of the wing. If True, only applies for y_ratio_tank_end =1.0 in the initial .xml file.

In [ ]:
from fastga.models.modify_config import ComputeConfigMod


compute_modify = api_cs23.generate_block_analysis(
        ComputeConfigMod(span_mod=[span_length_multiplier, fixed_engine_position, fuel_added_part]),
        [],
        str(AIRCRAFT_SPAN_FILE),
        True,
    )

output = compute_modify({})

In [ ]:
from fastga.models.modify_config.update_XML import UpdateXML


compute_update = api_cs23.generate_block_analysis(
        UpdateXML(span_mod=[span_length_multiplier, fixed_engine_position, fuel_added_part]),
        [],
        str(AIRCRAFT_SPAN_FILE),
        True,
    )

output = compute_update({})

In [ ]:
from fastga.models.geometry import GeometryFixedTailDistance as Geometry
from fastga.models.aerodynamics.aerodynamics import Aerodynamics
from fastga.models.weight.weight import Weight
from fastga.models.weight.mass_breakdown.update_mtow import UpdateMTOW
from fastga.models.performances.mission import Mission
from fastga.models.load_analysis.loads import Loads
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities

# engine_id = "fastga.wrapper.propulsion.basic_turboprop"
engine_id = "fastga.wrapper.propulsion.basicIC_engine"

# GEOMETRY
compute_geometry = api_cs23.generate_block_analysis(
        Geometry(propulsion_id=engine_id),
        [],
        str(AIRCRAFT_SPAN_FILE),
        True,
    )

output = compute_geometry({})


# AERODYNAMICS
compute_aero = api_cs23.generate_block_analysis(
        Aerodynamics(
            propulsion_id=engine_id,
            use_openvsp=True,
            compute_mach_interpolation=True,
            compute_slipstream_cruise=True,
#             wing_airfoil_file="naca43013_3.af"
        ),
        [],
        str(AIRCRAFT_REF_FILE),
        True,
    )

output = compute_aero({})


# WEIGHT
compute_weight = api_cs23.generate_block_analysis(
        Weight(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_SPAN_FILE),
        True,
    )

compute_weight_mod = api_cs23.generate_block_analysis(
        Weight(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_ref = compute_weight_ref({})
output_mod = compute_weight_mod({})


# MTOW
var_inputs = ["data:mission:sizing:fuel"]

compute_mtow_ref = api_cs23.generate_block_analysis(
        UpdateMTOW(),
        var_inputs,
        str(AIRCRAFT_REF_FILE),
        True,
    )

compute_mtow_mod = api_cs23.generate_block_analysis(
        UpdateMTOW(),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )


output_ref = compute_mtow_ref(inputs_dict)
output_mod = compute_mtow_mod(inputs_dict)